# Package loading and basic configurations

In [6]:
%load_ext autoreload
%autoreload 2

# load dependencies'
import pandas as pd
import geopandas as gpd

from envirocar import TrackAPI, DownloadClient, BboxSelector, ECConfig, TrackConverter
from envirocar import inspection as inspect
from envirocar import manipulation as manipulate


# create an initial but optional config and an api client
config = ECConfig()
track_api = TrackAPI(api_client=DownloadClient(config=config))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Querying enviroCar Tracks

The following cell queries tracks from the enviroCar API. It defines a bbox for the area of Gievenbeck, Münster (Germany) and requests 50 tracks. The result is a GeoDataFrame, which is a geo-extended Pandas dataframe from the GeoPandas library. It contains all information of the track in a flat dataframe format including a specific geometry column. 

In [2]:
bbox =BboxSelector([
    7.004130554199218, # min_x
    52.50590322041212, # min_y
    7.990351104736328, # max_x
    52.99874790276371  # max_y
])


# issue a query
track_df = track_api.get_tracks(bbox=bbox, num_results=10) # requesting 10 tracks inside the bbox
# track_df

In [3]:
track_df

,id,time,geometry,GPS Accuracy.value,GPS Accuracy.unit,Intake Pressure.value,Intake Pressure.unit,GPS Altitude.value,GPS Altitude.unit,GPS VDOP.value,...,sensor.id,sensor.fuelType,sensor.constructionYear,sensor.manufacturer,O2 Lambda Current.value,O2 Lambda Current.unit,O2 Lambda Current ER.value,O2 Lambda Current ER.unit,MAF.value,MAF.unit
0,5e08bc845bc8db428964203f,2019-12-29T12:11:19+00:00,POINT (7.27089 53.52839),1.899800,%,36.000001,kPa,45.999999,m,0.800000,...,58395f40e4b0a979d45bd61b,gasoline,2007,Dodge,NaN,NaN,NaN,NaN,NaN,NaN
1,5e08bc845bc8db4289642041,2019-12-29T12:11:25+00:00,POINT (7.27071 53.52830),2.033838,%,56.470588,kPa,44.999999,m,0.800000,...,58395f40e4b0a979d45bd61b,gasoline,2007,Dodge,NaN,NaN,NaN,NaN,NaN,NaN
2,5e08bc845bc8db4289642042,2019-12-29T12:11:30+00:00,POINT (7.27037 53.52828),1.500000,%,35.000000,kPa,44.000000,m,0.800000,...,58395f40e4b0a979d45bd61b,gasoline,2007,Dodge,NaN,NaN,NaN,NaN,NaN,NaN
3,5e08bc845bc8db4289642043,2019-12-29T12:11:35+00:00,POINT (7.27023 53.52824),2.000000,%,38.000001,kPa,44.999999,m,0.610282,...,58395f40e4b0a979d45bd61b,gasoline,2007,Dodge,NaN,NaN,NaN,NaN,NaN,NaN
4,5e08bc845bc8db4289642044,2019-12-29T12:11:40+00:00,POINT (7.27023 53.52824),2.500000,%,41.190981,kPa,45.000001,m,0.600000,...,58395f40e4b0a979d45bd61b,gasoline,2007,Dodge,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,580c50f7e4b01fb1c08ccb8c,2016-09-17T13:58:16+00:00,POINT (7.17977 52.58923),3.000000,%,223.036897,kPa,21.999999,m,NaN,...,523c72d1e4b060b8865cf1d4,diesel,2012,BMW,-34.475567,A,1.641869,ratio,115.590889,l/s
448,580c50f7e4b01fb1c08ccb8d,2016-09-17T13:58:21+00:00,POINT (7.18006 52.59104),3.000000,%,227.666673,kPa,21.999999,m,NaN,...,523c72d1e4b060b8865cf1d4,diesel,2012,BMW,-14.110937,A,1.602079,ratio,125.989998,l/s
449,580c50f7e4b01fb1c08ccb8e,2016-09-17T13:58:26+00:00,POINT (7.18047 52.59288),3.000000,%,215.999998,kPa,20.000000,m,NaN,...,523c72d1e4b060b8865cf1d4,diesel,2012,BMW,5.476563,A,1.838052,ratio,112.779157,l/s
450,580c50f7e4b01fb1c08ccb8f,2016-09-17T13:58:31+00:00,POINT (7.18097 52.59479),3.000000,%,231.000000,kPa,19.000001,m,NaN,...,523c72d1e4b060b8865cf1d4,diesel,2012,BMW,25.080614,A,1.629606,ratio,134.987064,l/s


## Aggregation

### Spatio-temporal aggregation based on weekday and hour

In [7]:
#Spatio temporally aggregate CO2 mean value with 1000 m of grid side
grid,subGrid=manipulate.spatioTemporalAggregation(track_df,"Speed.value","mean",1000)

AttributeError: module 'envirocar.EDA.manipulation' has no attribute 'spatioTemporalAggregation'

In [ ]:
#Visualize spatio-temporal aggregate
Visualiser.MosaicPlot(grid,subGrid,'Speed.value_mean')

### Grid wise Spatial Aggregation

In [ ]:
summary=Visualiser.aggregateByGrid(track_df,"Speed.value","mean",1000)

In [ ]:
choropleth=Visualiser.plotAggregate(summary,"mean_Speed.value")
choropleth

### Temporal Aggregation on hourly basis

In [ ]:
#Aggregate data temporally over the whole dataset
import seaborn as sns
x=Visualiser.aggregateHourly(track_df,'Speed.value','mean')
dayhourPivot=x.pivot(index='hour_',columns='weekday_',values='Speed.value_mean')
dayhourPivot
axs=sns.heatmap(dayhourPivot)

### Cluster  origin of tracks using dbscan algorithm

In [ ]:
#Get origin and destination of tracks
origin,destination=Visualiser.OriginAndDestination(track_df)

In [ ]:
#Get cluster centers
clusterCentres=Visualiser.getClusters(origin,3,2)

### Show clusters of track origins with bubblemap

In [ ]:
#Show cluster for origin of tracks overlaid with track points
clus=Visualiser.showClusters(clusterCentres,track_df)
clus